In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from datetime import datetime

In [2]:
# 🔹 1. Load Dataset
df = pd.read_csv("preprocessed_train_data.csv")

In [3]:
df

,year_of_purchase,age_of_vehicle,odometer_reading,last_service_date,last_service_type,number_of_services,last_service_kms,avg_kms_per_month,next_service_due_kms,next_service_due_date,...,service_center_Velachery,warranty_status_Active,warranty_status_Expired,insurance_status_Active,insurance_status_Expired,AMC_status_No,AMC_status_Yes,preferred_language_English,preferred_language_Hindi,preferred_language_Tamil
0,2019,-0.247899,-0.206041,2024-10-04,Minor,0.560456,56063,811,66063,2025-02-04,...,0,0,1,0,1,1,0,0,0,1
1,2019,-0.247899,-1.082204,2025-05-01,Major,-0.907667,30554,443,40554,2025-10-28,...,0,0,1,1,0,1,0,0,0,1
2,2020,-0.692162,-0.519720,2025-04-03,Minor,0.927486,46881,813,56881,2025-09-30,...,0,0,1,0,1,1,0,1,0,0
3,2019,-0.247899,0.601719,2024-11-18,Minor,0.193425,82509,1149,92509,2025-05-17,...,0,0,1,0,1,1,0,1,0,0
4,2015,1.529153,0.398341,2024-12-27,Major,-0.540636,73001,642,83001,2025-06-25,...,0,0,1,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2015,1.529153,1.174563,2025-02-16,Major,-0.907667,99377,838,109377,2025-08-15,...,0,0,1,1,0,0,1,0,1,0
996,2016,1.084890,0.923895,2024-10-14,Minor,-0.907667,90539,860,100539,2025-12-04,...,1,0,1,1,0,0,1,0,0,1
997,2021,-1.136425,-0.416348,2025-04-14,Minor,-0.907667,47837,1076,57837,2025-11-10,...,0,0,1,1,0,1,0,0,0,1
998,2015,1.529153,0.031489,2024-08-26,Minor,1.294517,64525,549,74525,2025-02-22,...,0,0,1,1,0,1,0,0,0,1


In [4]:
# 🔹 2. Date Handling
date_cols = ['last_service_date']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')
    df[f"{col}_days_ago"] = (datetime.today() - df[col]).dt.days
    df.drop(columns=[col], inplace=True)

In [5]:
df

,year_of_purchase,age_of_vehicle,odometer_reading,last_service_type,number_of_services,last_service_kms,avg_kms_per_month,next_service_due_kms,next_service_due_date,next_service_due_days,...,warranty_status_Active,warranty_status_Expired,insurance_status_Active,insurance_status_Expired,AMC_status_No,AMC_status_Yes,preferred_language_English,preferred_language_Hindi,preferred_language_Tamil,last_service_date_days_ago
0,2019,-0.247899,-0.206041,Minor,0.560456,56063,811,66063,2025-02-04,357,...,0,1,0,1,1,0,0,0,1,291
1,2019,-0.247899,-1.082204,Major,-0.907667,30554,443,40554,2025-10-28,296,...,0,1,1,0,1,0,0,0,1,82
2,2020,-0.692162,-0.519720,Minor,0.927486,46881,813,56881,2025-09-30,210,...,0,1,0,1,1,0,1,0,0,110
3,2019,-0.247899,0.601719,Minor,0.193425,82509,1149,92509,2025-05-17,180,...,0,1,0,1,1,0,1,0,0,246
4,2015,1.529153,0.398341,Major,-0.540636,73001,642,83001,2025-06-25,180,...,0,1,1,0,0,1,0,1,0,207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2015,1.529153,1.174563,Major,-0.907667,99377,838,109377,2025-08-15,180,...,0,1,1,0,0,1,0,1,0,156
996,2016,1.084890,0.923895,Minor,-0.907667,90539,860,100539,2025-12-04,180,...,0,1,1,0,0,1,0,0,1,281
997,2021,-1.136425,-0.416348,Minor,-0.907667,47837,1076,57837,2025-11-10,180,...,0,1,1,0,1,0,0,0,1,99
998,2015,1.529153,0.031489,Minor,1.294517,64525,549,74525,2025-02-22,180,...,0,1,1,0,1,0,0,0,1,330


In [6]:
# 🔹 3. Drop/Transform Columns
# Conditional drops in case some fields don't exist
drop_cols = ['customer_id', 'name', 'email', 'mobile_number', 'follow_up_date']
df = df.drop(columns=[col for col in drop_cols if col in df.columns], errors='ignore')

In [7]:
df

,year_of_purchase,age_of_vehicle,odometer_reading,last_service_type,number_of_services,last_service_kms,avg_kms_per_month,next_service_due_kms,next_service_due_date,next_service_due_days,...,warranty_status_Active,warranty_status_Expired,insurance_status_Active,insurance_status_Expired,AMC_status_No,AMC_status_Yes,preferred_language_English,preferred_language_Hindi,preferred_language_Tamil,last_service_date_days_ago
0,2019,-0.247899,-0.206041,Minor,0.560456,56063,811,66063,2025-02-04,357,...,0,1,0,1,1,0,0,0,1,291
1,2019,-0.247899,-1.082204,Major,-0.907667,30554,443,40554,2025-10-28,296,...,0,1,1,0,1,0,0,0,1,82
2,2020,-0.692162,-0.519720,Minor,0.927486,46881,813,56881,2025-09-30,210,...,0,1,0,1,1,0,1,0,0,110
3,2019,-0.247899,0.601719,Minor,0.193425,82509,1149,92509,2025-05-17,180,...,0,1,0,1,1,0,1,0,0,246
4,2015,1.529153,0.398341,Major,-0.540636,73001,642,83001,2025-06-25,180,...,0,1,1,0,0,1,0,1,0,207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2015,1.529153,1.174563,Major,-0.907667,99377,838,109377,2025-08-15,180,...,0,1,1,0,0,1,0,1,0,156
996,2016,1.084890,0.923895,Minor,-0.907667,90539,860,100539,2025-12-04,180,...,0,1,1,0,0,1,0,0,1,281
997,2021,-1.136425,-0.416348,Minor,-0.907667,47837,1076,57837,2025-11-10,180,...,0,1,1,0,1,0,0,0,1,99
998,2015,1.529153,0.031489,Minor,1.294517,64525,549,74525,2025-02-22,180,...,0,1,1,0,1,0,0,0,1,330


In [8]:
# 🔹 4. Handle Missing Values
df = df.dropna(subset=['next_service_due_days'])  # Drop rows where target is missing

In [9]:
# 🔹 5. Encode Categorical Columns
categorical_cols = df.select_dtypes(include='object').columns.drop(['pending_service'], errors='ignore')
df = pd.get_dummies(df, columns=categorical_cols)

In [10]:
# 🔹 6. Prepare Features for Regression
y_reg = df['next_service_due_days']
X_reg = df.drop(columns=['next_service_due_days', 'pending_service'], errors='ignore')

In [11]:
# 🔹 7. Split and Scale
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

In [12]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reg)
X_test_scaled = scaler.transform(X_test_reg)

In [13]:
import warnings
warnings.filterwarnings("ignore")

# ✅ 8. Random Forest Regressor
print("🌲 Random Forest Regressor")
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train_scaled, y_train_reg)
rf_preds = rf_reg.predict(X_test_scaled)

rf_mae = mean_absolute_error(y_test_reg, rf_preds)
rf_r2 = r2_score(y_test_reg, rf_preds)
print(f"MAE: {rf_mae:.2f}")
print(f"R² Score: {rf_r2:.2f}")

🌲 Random Forest Regressor
MAE: 16.65
R² Score: 0.79


In [14]:
# ✅ 9. Linear Regression
print("📈 Linear Regression")
lr_reg = LinearRegression()
lr_reg.fit(X_train_scaled, y_train_reg)
lr_preds = lr_reg.predict(X_test_scaled)

lr_mae = mean_absolute_error(y_test_reg, lr_preds)
lr_r2 = r2_score(y_test_reg, lr_preds)
print(f"MAE: {lr_mae:.2f}")
print(f"R² Score: {lr_r2:.2f}")

📈 Linear Regression
MAE: 15.41
R² Score: 0.84


In [15]:
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# (Assuming you have already preprocessed X_reg and y_reg)
X_train, X_test, y_train, y_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Fit your regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Save the model with pickle
filename = 'final_regression_model.sav'
pickle.dump(model, open(filename, 'wb'))

# ===============================
# Later: To Load and Use the Model
# ===============================
loaded_model = pickle.load(open(filename, 'rb'))
predictions = loaded_model.predict(X_test)

In [16]:
import pickle

# Save the trained model to a file
with open("final_regression_model.pkl", "wb") as file:
    pickle.dump(model, file)
# Load model
with open("final_regression_model.pkl", "rb") as file:
    loaded_model = pickle.load(file)

# Make predictions
preinput = X_test[:5]  # Example input
result = loaded_model.predict(preinput)

print("Prediction:", result)

Prediction: [180.00000001 123.00000001 151.         175.44312512 180.        ]
